# Desplegar backend en HuggingFace


En este post vamos a ver cómo desplegar un backend en HuggingFace. Vamos a ver cómo hacerlo de dos maneras, mediante la forma común, creando una aplicación con Gradio, y mediante un opción diferente usando FastAPI, Langchain y Docker

Para ambos casos va a ser necesario tener una cuenta en HuggingFace, ya que vamos a desplegar el backend en un space de HuggingFace.

## Desplegar backend con Gradio

### Crear space

Primero de todo, creamos un nuevo espacio en HuggingFace.

 * Ponemos un nombre, una descripción y elegimos la licencia.
 * Elegimos Gradio como el tipo de SDK. Al elegir Gradio, nos aparecerán plantillas, así que elegimos la plantilla de chatbot.
 * Seleccionamos el HW en el que vamos a desplegar el backend, yo voy a elegir la CPU gratuita, pero tú elige lo que mejor consideres.
 * Y por último hay que elegor si queremos crear el espacio público o privado.

![backend gradio - create space](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/backend-gradio-create-space.webp)

### Código

Al crear el space, podemos clonarlo o podemos ver los archivos en la propia página de HuggingFace. Podemos ver que se han creado 3 archivos, `app.py`, `requirements.txt` y `README.md`. Así que vamos a ver qué poner en cada uno

#### app.py

Aquí tenemos el código de la aplicación. Cómo hemos elegido la plantilla de chatbot, ya tenemos mucho hecho, pero vamos a tener que cambiar 2 cosas, primero el modelo de lenguaje y el system prompt

Como modelo de lenguaje veía ``HuggingFaceH4/zephyr-7b-beta``, pero vamos a usar ``Qwen/Qwen2.5-72B-Instruct``, que es un modelo muy capaz.

Así que busca el texto ``client = InferenceClient("HuggingFaceH4/zephyr-7b-beta")`` y reemplázalo por ``client = InferenceClient("Qwen/Qwen2.5-72B-Instruct")``, o espera que más adelante pondré todo el código.

También vamos a cambiar el system prompt, que por defecto es ``You are a friendly Chatbot.``, pero como es un modelo entrenado en su mayoría en inglés, es probable que si le hablas en otro idioma te responda en inglés, así que vamos a cambiarlo por ``You are a friendly Chatbot. Always reply in the language in which the user is writing to you.``.

Así que busca el texto ``gr.Textbox(value="You are a friendly Chatbot.", label="System message"),`` y reemplázalo por ``gr.Textbox(value="You are a friendly Chatbot. Always reply in the language in which the user is writing to you.", label="System message"),``, o espera a que ahora voy a poner todo el código.

``` python
import gradio as gr
from huggingface_hub import InferenceClient

"""
For more information on `huggingface_hub` Inference API support, please check the docs: https://huggingface.co/docs/huggingface_hub/v0.22.2/en/guides/inference
"""
client = InferenceClient("Qwen/Qwen2.5-72B-Instruct")


def respond(
    message,
    history: list[tuple[str, str]],
    system_message,
    max_tokens,
    temperature,
    top_p,
):
    messages = [{"role": "system", "content": system_message}]

    for val in history:
        if val[0]:
            messages.append({"role": "user", "content": val[0]})
        if val[1]:
            messages.append({"role": "assistant", "content": val[1]})

    messages.append({"role": "user", "content": message})

    response = ""

    for message in client.chat_completion(
        messages,
        max_tokens=max_tokens,
        stream=True,
        temperature=temperature,
        top_p=top_p,
    ):
        token = message.choices[0].delta.content

        response += token
        yield response


"""
For information on how to customize the ChatInterface, peruse the gradio docs: https://www.gradio.app/docs/chatinterface
"""
demo = gr.ChatInterface(
    respond,
    additional_inputs=[
        gr.Textbox(value="You are a friendly Chatbot. Always reply in the language in which the user is writing to you.", label="System message"),
        gr.Slider(minimum=1, maximum=2048, value=512, step=1, label="Max new tokens"),
        gr.Slider(minimum=0.1, maximum=4.0, value=0.7, step=0.1, label="Temperature"),
        gr.Slider(
            minimum=0.1,
            maximum=1.0,
            value=0.95,
            step=0.05,
            label="Top-p (nucleus sampling)",
        ),
    ],
)


if __name__ == "__main__":
    demo.launch()
```

#### requirements.txt

Este es el archivo en el que estarán escritas las dependencias, pero para este caso va a ser muy sencillo:

``` txt
huggingface_hub==0.25.2
```

#### README.md

Este es el archivo en el que vamos a poner la información del espacio. En los spaces de HuggingFace, al inicio de los readmes, se pone un código para que HuggingFace sepa cómo mostrar la miniatura del espacio, qué fichero tiene que usar para ejecutar el código, version del sdk, etc.

``` md
---
title: SmolLM2
emoji: 💬
colorFrom: yellow
colorTo: purple
sdk: gradio
sdk_version: 5.0.1
app_file: app.py
pinned: false
license: apache-2.0
short_description: Gradio SmolLM2 chat
---

An example chatbot using [Gradio](https://gradio.app), [`huggingface_hub`](https://huggingface.co/docs/huggingface_hub/v0.22.2/en/index), and the [Hugging Face Inference API](https://huggingface.co/docs/api-inference/index).
```

### Despliegue

Si hemos clonado el espacio, tenemos que hacer un commit y un push. Si hemos modificado los archivos en HuggingFace, con guardarlos es suficiente.

Así que cuando estén los cambios en HuggingFace, tendremos que esperar unos segundos para que se construya el espacio y podremos usarlo.

![backend gradio - chatbot](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/backend-gradio-chatbot.webp)

### Backend

Muy bien, hemos hecho un chatbot, pero no era la intención, aquí habíamos venido a hacer un backend! Para, para, fíjate lo que pone debajo del chatbot

![backend gradio - Use via API](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/backend-gradio-chatbot-edited.webp)

Podemos ver una texto ``Use via API``, donde si pulsamos se nos abre un menú con una API para poder usar el chatbot.

![backend gradio - API](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/backend%20gradio%20-%20API.webp)

Vemos que nos da una documentación de cómo usar la API, tanto con Python, con JavaScript, como con bash.

### Prueba de la API

Usamos el código de ejemplo de Python.

In [10]:
from gradio_client import Client

client = Client("Maximofn/SmolLM2")
result = client.predict(
		message="Hola, ¿cómo estás? Me llamo Máximo",
		system_message="You are a friendly Chatbot. Always reply in the language in which the user is writing to you.",
		max_tokens=512,
		temperature=0.7,
		top_p=0.95,
		api_name="/chat"
)
print(result)

Loaded as API: https://maximofn-smollm2.hf.space ✔
¡Hola Máximo! Mucho gusto, estoy bien, gracias por preguntar. ¿Cómo estás tú? ¿En qué puedo ayudarte hoy?


Estamos haciendo llamadas a la API de `InferenceClient` de HuggingFace, así que podríamos pensar, ¿Para qué hemos hecho un backend, si podemos llamar directamente a la API de HuggingFace? Pues lo vas a ver a continuación.

In [ ]:
result = client.predict(
		message="¿Cómo me llamo?",
		system_message="You are a friendly Chatbot. Always reply in the language in which the user is writing to you.",
		max_tokens=512,
		temperature=0.7,
		top_p=0.95,
		api_name="/chat"
)
print(result)

Tu nombre es Máximo. ¿Es correcto?


La plantilla de chat de Gradio maneja el historial por nosotros, de manera que cada vez que creamos un nuevo `cliente`, se crea un nuevo hilo de conversación.

Vamos a probar a crear un nuevo cliente, y ver si se crea un nuevo hilo de conversación.

In [12]:
from gradio_client import Client

new_client = Client("Maximofn/SmolLM2")
result = new_client.predict(
		message="Hola, ¿cómo estás? Me llamo Luis",
		system_message="You are a friendly Chatbot. Always reply in the language in which the user is writing to you.",
		max_tokens=512,
		temperature=0.7,
		top_p=0.95,
		api_name="/chat"
)
print(result)

Loaded as API: https://maximofn-smollm2.hf.space ✔
Hola Luis, estoy muy bien, gracias por preguntar. ¿Cómo estás tú? Es un gusto conocerte. ¿En qué puedo ayudarte hoy?


Ahora le volvemos a preguntar cómo me llamo

In [13]:
result = new_client.predict(
		message="¿Cómo me llamo?",
		system_message="You are a friendly Chatbot. Always reply in the language in which the user is writing to you.",
		max_tokens=512,
		temperature=0.7,
		top_p=0.95,
		api_name="/chat"
)
print(result)

Te llamas Luis. ¿Hay algo más en lo que pueda ayudarte?


Como vemos, tenemos dos clientes, cada uno con su propio hilo de conversación.

## Desplegar backend con FastAPI, Langchain y Docker

Ahora vamos a hacer lo mismo, crear un backend de un chatbot, con el mismo modelo, pero en este caso usando FastAPI, Langchain y Docker.

### Crear space

Tenemos que crear un nuevo espacio, pero en este caso lo vamos a hacer de otra manera

 * Ponemos un nombre, una descripción y elegimos la licencia.
 * Elegimos Docker como el tipo de SDK. Al elegir Docker, nos aparecerán plantillas, así que elegimos una plantilla en blanco.
 * Seleccionamos el HW en el que vamos a desplegar el backend, yo voy a elegir la CPU gratuita, pero tú elige lo que mejor consideres.
 * Y por último hay que elegor si queremos crear el espacio público o privado.

![backend docker - create space](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/backend-docker-create-space.webp)

### Código

Ahora, al crear el space, vemos que solo tenemos un archivo, el ``README.md``. Así que vamos a tener que crear todo el código nosotros.

#### app.py

Vamos a crear el código de la aplicación

Empezamos con las librerías necesarias

``` python
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from huggingface_hub import InferenceClient

from langchain_core.messages import HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

import os
from dotenv import load_dotenv
load_dotenv()
```

Cargamos `fastapi` para poder crear las rutas de la API, `pydantic` para crear la plantilla de las querys, `huggingface_hub` para poder crear un modelo de lengiaje, `langchain` para poder indicarle al modelo si los mensajes son del chatbot o del usuario y `langgraph` para poder crear el chatbot.

Además cargamos `os` y `dotenv` para poder cargar las variables de entorno.

Cargamos el token de HuggingFace

``` python
# HuggingFace token
HUGGINGFACE_TOKEN = os.environ.get("HUGGINGFACE_TOKEN", os.getenv("HUGGINGFACE_TOKEN"))
```

Creamos el modelo de lenguaje

``` python
model = InferenceClient(
    model="Qwen/Qwen2.5-72B-Instruct",
    api_key=os.getenv("HUGGINGFACE_TOKEN")
)
```

Creamos ahora un función para llamar al modelo

``` python
def call_model(state: MessagesState):
    """
    Call the model with the given messages

    Args:
        state: MessagesState

    Returns:
        dict: A dictionary containing the generated text and the thread ID
    """
    # Convert LangChain messages to HuggingFace format
    hf_messages = []
    for msg in state["messages"]:
        if isinstance(msg, HumanMessage):
            hf_messages.append({"role": "user", "content": msg.content})
        elif isinstance(msg, AIMessage):
            hf_messages.append({"role": "assistant", "content": msg.content})
    
    # Call the API
    response = model.chat_completion(
        messages=hf_messages,
        temperature=0.5,
        max_tokens=64,
        top_p=0.7
    )
    
    # Convert the response to LangChain format
    ai_message = AIMessage(content=response.choices[0].message.content)
    return {"messages": state["messages"] + [ai_message]}
```

Convertimos los mensajes de formato LangChain a formato HuggingFace, así podemos usar el modelo de lenguaje.

Definimos un plantilla para las querys

``` python
class QueryRequest(BaseModel):
    query: str
    thread_id: str = "default"
```

Las querys van a tener un `query`, el mensaje del usuario, y un `thread_id`, que es el identificador del hilo de la conversación y más adelante explicaremos para qué lo usamos.

Creamos un grafo de LangGraph

``` python
# Define the graph
workflow = StateGraph(state_schema=MessagesState)

# Define the node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
graph_app = workflow.compile(checkpointer=memory)
```

Con esto lo que hacemos es crear un grafo de LangGraph, que es una estructura de datos que nos permite crear un chatbot y que gestiona por nosotros el estado del chatbot, es decir, entre otras cosas, el historial de mensajes. Así no lo tenemos que hacer nosotros.

Creamos la aplicación de FastAPI

``` python
app = FastAPI(title="LangChain FastAPI", description="API to generate text using LangChain and LangGraph")
```

Creamos los endpoints de la API

``` python
# Welcome endpoint
@app.get("/")
async def api_home():
    """Welcome endpoint"""
    return {"detail": "Welcome to FastAPI, Langchain, Docker tutorial"}

# Generate endpoint
@app.post("/generate")
async def generate(request: QueryRequest):
    """
    Endpoint to generate text using the language model
    
    Args:
        request: QueryRequest
        query: str
        thread_id: str = "default"

    Returns:
        dict: A dictionary containing the generated text and the thread ID
    """
    try:
        # Configure the thread ID
        config = {"configurable": {"thread_id": request.thread_id}}
        
        # Create the input message
        input_messages = [HumanMessage(content=request.query)]
        
        # Invoke the graph
        output = graph_app.invoke({"messages": input_messages}, config)
        
        # Get the model response
        response = output["messages"][-1].content
        
        return {
            "generated_text": response,
            "thread_id": request.thread_id
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error al generar texto: {str(e)}")
```

Hemos creado el endpoint `/` que nos devolverá un texto cuando accedamos a la API, y el endpoint `/generate` que es el que usaremos para generar el texto.

Si nos fijamos en la función `generate` tenemos la variable `config`, que es un diccionario que contiene el `thread_id`. Este `thread_id` es el que nos permite tener un historial de mensajes de cada usuario, de esta manera, diferentes usuarios pueden usar el mismo endpoint y tener su propio historial de mensajes.

Por último, tenemos el código para que se pueda ejecutar la aplicación

``` python
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
```

Vamos a escribir todo el código junto

``` python
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from huggingface_hub import InferenceClient

from langchain_core.messages import HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

import os
from dotenv import load_dotenv
load_dotenv()

HUGGINGFACE_TOKEN = os.environ.get("HUGGINGFACE_TOKEN", os.getenv("HUGGINGFACE_TOKEN"))

# Initialize the HuggingFace model
model = InferenceClient(
    model="Qwen/Qwen2.5-72B-Instruct",
    api_key=os.getenv("HUGGINGFACE_TOKEN")
)

# Define the function that calls the model
def call_model(state: MessagesState):
    """
    Call the model with the given messages

    Args:
        state: MessagesState

    Returns:
        dict: A dictionary containing the generated text and the thread ID
    """
    # Convert LangChain messages to HuggingFace format
    hf_messages = []
    for msg in state["messages"]:
        if isinstance(msg, HumanMessage):
            hf_messages.append({"role": "user", "content": msg.content})
        elif isinstance(msg, AIMessage):
            hf_messages.append({"role": "assistant", "content": msg.content})
    
    # Call the API
    response = model.chat_completion(
        messages=hf_messages,
        temperature=0.5,
        max_tokens=64,
        top_p=0.7
    )
    
    # Convert the response to LangChain format
    ai_message = AIMessage(content=response.choices[0].message.content)
    return {"messages": state["messages"] + [ai_message]}

# Define the graph
workflow = StateGraph(state_schema=MessagesState)

# Define the node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
graph_app = workflow.compile(checkpointer=memory)

# Define the data model for the request
class QueryRequest(BaseModel):
    query: str
    thread_id: str = "default"

# Create the FastAPI application
app = FastAPI(title="LangChain FastAPI", description="API to generate text using LangChain and LangGraph")

# Welcome endpoint
@app.get("/")
async def api_home():
    """Welcome endpoint"""
    return {"detail": "Welcome to FastAPI, Langchain, Docker tutorial"}

# Generate endpoint
@app.post("/generate")
async def generate(request: QueryRequest):
    """
    Endpoint to generate text using the language model
    
    Args:
        request: QueryRequest
        query: str
        thread_id: str = "default"

    Returns:
        dict: A dictionary containing the generated text and the thread ID
    """
    try:
        # Configure the thread ID
        config = {"configurable": {"thread_id": request.thread_id}}
        
        # Create the input message
        input_messages = [HumanMessage(content=request.query)]
        
        # Invoke the graph
        output = graph_app.invoke({"messages": input_messages}, config)
        
        # Get the model response
        response = output["messages"][-1].content
        
        return {
            "generated_text": response,
            "thread_id": request.thread_id
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error al generar texto: {str(e)}")

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
```